In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
%cd ..

In [4]:
from src.dataset import BraTSDataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import yaml

# Baseline augmentation

In [11]:
from src.models.baseline import (
    BaselineInpaintingGenerator,
    BaselineGradeSegmenter,
    BaselineShapeSegmenter,
)

def map_grade(x: torch.Tensor, requires_grad: bool = False):
    out = torch.zeros(x.shape, dtype=torch.float, requires_grad=requires_grad).to(
        x.device
    )
    out[(x == 1).nonzero(as_tuple=True)] = 0.5
    out[(x == 2).nonzero(as_tuple=True)] = 0.75
    out[(x == 3).nonzero(as_tuple=True)] = 1.0
    return out

BASELINE_INP_GEN_CP_NAME = "network_inpainting_gen.pth.tar"
BASELINE_SHAPE_CP_NAME = "network_shape.pth.tar"
BASELINE_GRADE_CP_NAME = "network_grade.pth.tar"

In [6]:
inp_gen_cp_path = f"results/baseline/21-01-2022 16-44-21/checkpoints/100/{BASELINE_INP_GEN_CP_NAME}"
shape_cp_path = f"results/baseline/20-01-2022 10-20-16/checkpoints/200/{BASELINE_SHAPE_CP_NAME}"
grade_cp_path = f"results/baseline/20-01-2022 10-20-16/checkpoints/200/{BASELINE_GRADE_CP_NAME}"

device = torch.device(f"cuda:{torch.cuda.current_device()}") if torch.cuda.is_available() else "cpu"
# device = "cpu"

shape_cp = torch.load(shape_cp_path)
shape_seg = BaselineShapeSegmenter().to(device)
shape_seg.load_state_dict(shape_cp["state_dict"])

grade_cp = torch.load(grade_cp_path)
grade_seg = BaselineGradeSegmenter().to(device)
grade_seg.load_state_dict(grade_cp["state_dict"])

inp_gen_cp = torch.load(inp_gen_cp_path)
inp_gen = BaselineInpaintingGenerator().to(device)
inp_gen.load_state_dict(inp_gen_cp["state_dict"])

<All keys matched successfully>

In [7]:
input_path = "resources/data/brats20/validation"
fixed_indices = "resources/indices/brats20_25p.yml"
with open(fixed_indices) as file:
    indices = yaml.safe_load(file)
batch_size = 8

aug_dataset = BraTSDataset(root=input_path, mode="test", fixed_indices=indices["val"])
aug_loader = DataLoader(dataset=aug_dataset, batch_size=batch_size, shuffle=False)

In [12]:
pb = tqdm(aug_loader)
for batch in pb:
    t_t1 = batch["t1"].to(device)
    t_t1ce = batch["t1ce"].to(device)
    t_t2 = batch["t2"].to(device)
    t_flair = batch["flair"].to(device)

    t_m_brain = batch["mask_brain_shape"].to(device)
    t_m_shape = batch["mask_tumor_shape"].to(device)
    t_m_grade = batch["mask_tumor_grade"].to(device)
    t_m_grade_mapped = map_grade(batch["mask_tumor_grade"]).to(device)
    t_m_circles = batch["mask_circles"].to(device)

    t_m_ed = batch["mask_ed"].to(device)
    t_m_et = batch["mask_et"].to(device)
    t_m_net = batch["mask_net"].to(device)
    # TODO implement
    print(t_m_ed.shape)
    break

  0%|          | 0/67 [00:00<?, ?it/s]

torch.Size([8, 1, 256, 256])
